In [26]:
import torch
from PIL import Image
import open_clip
import os

In [27]:
name = "ViT-B-16-plus-240"
pretrained = "laion400m_e32"
model, train_transform, eval_transform = open_clip.create_model_and_transforms(name, pretrained=pretrained)

In [28]:
from pathlib import Path

ouptut_dir = "openvino_fp32"
if not os.path.exists(ouptut_dir):
    os.makedirs(ouptut_dir)
    
with open(Path(ouptut_dir) / "model_index.txt", 'w') as fd:
    fd.write(f"{name}, {pretrained}\n")

In [29]:
tokenizer = open_clip.get_tokenizer('ViT-B-16-plus-240')

image = eval_transform(Image.open("../docs/CLIP.png")).unsqueeze(0)
text = tokenizer("a cat")

In [30]:
torch.onnx.export(
    model.visual,
    image,
    "image_encoder.onnx",  # where to save the model
    opset_version=14,  # the ONNX version to export the model to
    input_names=["image"],  # the model's input names
    output_names=["image_embedding"],  # the model's output names
    dynamic_axes={  # variable length axes
        "image": {0: "batch"},
        "image_embedding": {0: "batch"},
    }
)

In [31]:
from openvino.tools.mo import convert_model
import openvino.runtime as ov

ov_encoder = convert_model("image_encoder.onnx")
ov.serialize(ov_encoder, ouptut_dir + "/image_encoder.xml")

In [32]:
cast_dtype = model.transformer.get_cast_dtype()

x = model.token_embedding(text).to(cast_dtype)  # [batch_size, n_ctx, d_model]

x = x + model.positional_embedding.to(cast_dtype)
x = x.permute(1, 0, 2)  # NLD -> LND


In [33]:
torch.onnx.export(
    model.transformer,
    (x, model.attn_mask),
    "text_encoder.onnx",  # where to save the model
    opset_version=14,  # the ONNX version to export the model to
    input_names=["input_ids", "attention_mask"],  # the model's input names
    output_names=["text_embeds"],  # the model's output names
    dynamic_axes={  # variable length axes
        "input_ids": {1: "batch"}
    }
)

In [34]:
from openvino.tools.mo import convert_model
import openvino.runtime as ov

ov_transformer = convert_model("text_encoder.onnx")
ov.serialize(ov_transformer, ouptut_dir + "/text_encoder.xml")

In [35]:
!benchmark_app -m image_encoder.onnx -shape "image[1,3,240,240]" -api sync

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to LATENCY.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 598.39 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     image (node: image) : f32 / [...] / [?,3,240,240]
[ INFO ] Model outputs:
[ INFO ]     image_embedding (node: image_embedding) : f32 / [...] / [?,640]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[ INFO ] Reshaping model: '

In [36]:
!benchmark_app -m text_encoder.onnx -shape "input_ids[77,1,640],attention_mask[77,77]" -api sync

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to LATENCY.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 354.08 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : f32 / [...] / [77,?,640]
[ INFO ]     attention_mask (node: attention_mask) : f32 / [...] / [77,77]
[ INFO ] Model outputs:
[ INFO ]     text_embeds (node: text_embeds) : f32 / [...] / [77,?,640]
[Step 5/11] Resizing model to match image siz

In [37]:
torch.onnx.export(
    model,
    (image, text),
    "model.onnx",  # where to save the model
    opset_version=14,  # the ONNX version to export the model to
    input_names=["image", "text"],  # the model's input names
    output_names=["image_embedding"],  # the model's output names
    dynamic_axes={  # variable length axes
        "image": {0: "batch"},
        "text": {0: "batch"},
    }
)

In [38]:
from openvino.tools.mo import convert_model
import openvino.runtime as ov

ov_transformer = convert_model("model.onnx")
ov.serialize(ov_transformer, "model.xml")

In [39]:
!benchmark_app -m model.xml -shape "image[1,3,240,240],text[1,77]" -api sync

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to LATENCY.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 523.88 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     image (node: image) : f32 / [...] / [?,3,240,240]
[ INFO ]     text (node: text) : i64 / [...] / [?,77]
[ INFO ] Model outputs:
[ INFO ]     image_embedding (node: image_embedding) : f32 / [...] / [?,640]
[ INFO ]     3690 (node: 3690) : f32 / [...] / [?,640]
[ INFO ]   